In [67]:
import pandas as pd
sampleTripMilestones = pd.DataFrame.from_csv('./data/2014_SF_Top_TripMileStones.csv')
sampleTripMilestones
# help(sampleTripMilestones)
x = sampleTripMilestones.reset_index()
routeDict={}
for index, row in x.iterrows():
    routeInfo={}
    routeInfo['totalDistance'] = row['totalDistance']
    routeInfo['milestoneCount'] = row['milestoneCount']
    routeInfo['mileStoneLats'] = row['mileStoneLats']
    routeInfo['mileStoneLngs'] = row['mileStoneLngs']
    
    routeDict[(row['startStationId'], row['endStationId'])] = routeInfo

len(routeDict)


29

In [70]:
import datetime
import csv
import ast

def getTripTimeline(row):
    tripTimelineList=[]

    routeMileStoneDict=routeDict[(row['startStationId'], row['endStationId'])]
    milestoneCount =routeMileStoneDict['milestoneCount']
    routeDict[(row['startStationId'], row['endStationId'])]

    lats=ast.literal_eval(routeMileStoneDict['mileStoneLats'])
    lngs=ast.literal_eval(routeMileStoneDict['mileStoneLngs'])
    times=ast.literal_eval(row['mileStoneTimes'])

    curMileStoneIndex = 0
    while (curMileStoneIndex < milestoneCount):
        currentTimeLineEntry = {} 
        currentTimeLineEntry['time']= times[curMileStoneIndex]
        currentTimeLineEntry['lat']= lats[curMileStoneIndex]
        currentTimeLineEntry['lng']= lngs[curMileStoneIndex]
    
        tripTimelineList.append(currentTimeLineEntry)
        curMileStoneIndex = curMileStoneIndex + 1
    return tripTimelineList



In [73]:
import datetime
def getFilledInTimeline(tripTimelineList):
    filledInTimelineList=[]
    
    
    prevTime=''
    prevLat=''
    preLng=''
    for i,currentTimeLineEntry in enumerate(tripTimelineList):
        currentTime = datetime.datetime.strptime(currentTimeLineEntry['time'], "%m/%d/%y %H:%M:%S")
        if (i > 0):
            timedifference=(currentTime - prevTime).seconds
            missingStepCount=(timedifference/10 -1)
            
            curMissingStep=1;
            while (curMissingStep <= missingStepCount ):
                missingTimeLineEntry = {} 
                missingTime = prevTime +  datetime.timedelta(seconds=(curMissingStep *10))
                missingTimeLineEntry['time'] = missingTime.strftime("%m/%d/%y %H:%M:%S")

                startMilestoneLat = float(prevLat)
                startMilestoneLng = float(preLng)

                endMilestoneLat = float(currentTimeLineEntry['lat'])
                endMilestoneLng = float(currentTimeLineEntry['lng'])

                incrementalLat = ((endMilestoneLat - startMilestoneLat)*curMissingStep)/(missingStepCount+1)
                missingTimeLineEntry['lat'] = str(startMilestoneLat + incrementalLat)

                incrementalLng = ((endMilestoneLng - startMilestoneLng)*curMissingStep)/(missingStepCount+1)
                missingTimeLineEntry['lng'] = str(startMilestoneLng + incrementalLng)

                filledInTimelineList.append(missingTimeLineEntry)
                curMissingStep = curMissingStep + 1
        prevTime=currentTime
        prevLat=currentTimeLineEntry['lat']
        preLng=currentTimeLineEntry['lng']
            
            
        filledInTimelineList.append(currentTimeLineEntry)
    return filledInTimelineList


In [74]:

topRouteMileStoneTimes= pd.DataFrame.from_csv('./data/2014_SF_TripsOfTop30Routes_Aug2013_Milestones.csv')
topRouteMileStoneTimes


# print routeDict[(69, 39)]['milestoneCount']

toCSV=[]
for index, row in topRouteMileStoneTimes.iterrows():
    tripTimeline = getTripTimeline(row)
    filledInTheTimeline=getFilledInTimeline(tripTimeline)
    toCSV.extend(filledInTheTimeline)


In [75]:

# toCSV

keys=[
 'time',
 'lat',
 'lng']
with open('./data/2014_SF_Aug_TopTimeline.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys,delimiter=',')
    dict_writer.writeheader()
    dict_writer.writerows(toCSV)